## QuanA based MCQ Generator

In [1]:
%load_ext autoreload
%autoreload 2

import os
curpath = os.getcwd()
os.chdir(curpath.split("core")[0])

In [2]:
import openai
import json
import time

from prompts import SIMPLE_QUANT_MCQ_WITH_TE_PROMPT, SIMPLE_QUANT_MCQ_WITH_E_PROMPT, SIMPLE_QUANT_MCQ_WITH_T_PROMPT, CALCULATION_CHECK_PROMPT
from core.services.qna_gen.utils import add_dicts, calculate_cost_gpt4_8k
from core.services.qna_gen.helper import get_results_from_wolfram_cloud, get_results_from_wolfram_alpha

from dotenv import load_dotenv
load_dotenv()

True

### Numerical MCQs

In [3]:

def generate_quant_numerical_mcqs(theory=None, n=1, examples = None, model_id = "gpt-4-0613"):

    if theory is None and examples is None:
        raise Exception("Either context or example must be provided")

    elif theory is not None and examples is not None:
        conversation = [{"role": "system", "content": SIMPLE_QUANT_MCQ_WITH_TE_PROMPT}]

        user_prompt = f"""
        TOPIC_THEORY:
        //theory//

        {theory}

        //theory//
    
        N: {n}

        EXAMPLES:
        {str(examples)}

        QUESTIONS: 
        """

    elif theory is not None and examples is None:
        conversation = [{"role": "system", "content": SIMPLE_QUANT_MCQ_WITH_T_PROMPT}]

        user_prompt = f"""
        TOPIC_THEORY:
        //theory//

        {theory}

        //theory//
    
        N: {n}

        QUESTIONS: 
        """

    elif theory is None and examples is not None:
        conversation = [{"role": "system", "content": SIMPLE_QUANT_MCQ_WITH_E_PROMPT}]

        user_prompt = f"""
        EXAMPLES:
        {str(examples)}
    
        N: {n}

        QUESTIONS: 
        """

    user_message = {'role': 'user', "content": user_prompt}
    conversation.append(user_message)

    total_usage = {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}
    response = openai.ChatCompletion.create(
                                    model= model_id,
                                    messages = conversation,
                                    temperature = 1
                                    )
    
    total_usage = add_dicts(total_usage, dict(response.usage))

    output = response.choices[0].message.content

    return {"output": output, "total_usage": total_usage}


In [4]:
example = {
        "question": "A shopkeeper allows 4% discount and gives 1 article free on purchase of 15 article. He earns 35% profit during the transaction. By what percent above the cost price he marked goods?",
        "choices": ["60%", "50%", "44%", "30%"],
        "answer": "50%"
    }

example = {
        "question": "A candidate scores 25% and fails by 30 marks, while another candidate who scores 50% marks, gets 20 marks more than the minimum required marks to pass the examination. Find the maximum marks for the examination.",
        "choices": ["200", "120", "300", "350"],
        "answer": "200"
    }

examples = [{
        "question": "There are three inlet taps whose diameters are 1 cm, 3 cm and 5 cm respectively. The rate of flow of the water is directly proportional to the square of the diameter. It takes 7 minutes for the smallest pipe to fill an empty tank. Find the time taken to fill an empty tank when all the three taps are opened.",
        "choices": ["13 sec", "15 sec", "12 sec", "10 sec"],
        "answer": "12 sec"
    },
    {
        "question": "A candidate scores 25% and fails by 30 marks, while another candidate who scores 50% marks, gets 20 marks more than the minimum required marks to pass the examination. Find the maximum marks for the examination.",
        "choices": ["200", "120", "300", "350"],
        "answer": "200"
    }
]

# examples = [{
#         "question": "There are three inlet taps whose diameters are 1 cm, 3 cm and 5 cm respectively. The rate of flow of the water is directly proportional to the square of the diameter. It takes 7 minutes for the smallest pipe to fill an empty tank. Find the time taken to fill an empty tank when all the three taps are opened.",
#         "choices": ["13 sec", "15 sec", "12 sec", "10 sec"],
#         "answer": "12 sec"
#     }
# ]

output = generate_quant_numerical_mcqs(
                        # context = context,
                        n = 3,
                        examples = examples
                        )

print(output['output'])
print(output['total_usage'])

{
    "mcqs": [
        {
            "question": "A car traveling at a speed of 80 km/h uses 10 liters of fuel per hour. The car's fuel consumption is directly proportional to its speed. How much fuel would the car use in an hour if it was traveling at a speed of 120 km/h?",
            "choices": {"a": "12 liters", "b": "15 liters", "c": "18 liters", "d": "20 liters"},
            "answer": {"b": "15 liters"}
        },
        {
            "question": "In a class test of 50 marks, John scored 60% and he surpassed Sam by 5 marks. Sam scored 50% marks. What are the minimum passing marks in the test?",
            "choices": {"a": "20", "b": "25", "c": "30", "d": "35"},
            "answer": {"a": "20"}
        },
        {
            "question": "There are two outlets taps whose diameters are 2 cm and 4 cm respectively. The rate of flow of the water is directly proportional to the square of the diameter. It takes 6 minutes for the smallest pipe to empty a full tank. Find the time ta

In [5]:
{'completion_tokens': 314, 'total_tokens': 978, 'prompt_tokens': 664}

{'completion_tokens': 314, 'total_tokens': 978, 'prompt_tokens': 664}

In [6]:
output['output']

'{\n    "mcqs": [\n        {\n            "question": "A car traveling at a speed of 80 km/h uses 10 liters of fuel per hour. The car\'s fuel consumption is directly proportional to its speed. How much fuel would the car use in an hour if it was traveling at a speed of 120 km/h?",\n            "choices": {"a": "12 liters", "b": "15 liters", "c": "18 liters", "d": "20 liters"},\n            "answer": {"b": "15 liters"}\n        },\n        {\n            "question": "In a class test of 50 marks, John scored 60% and he surpassed Sam by 5 marks. Sam scored 50% marks. What are the minimum passing marks in the test?",\n            "choices": {"a": "20", "b": "25", "c": "30", "d": "35"},\n            "answer": {"a": "20"}\n        },\n        {\n            "question": "There are two outlets taps whose diameters are 2 cm and 4 cm respectively. The rate of flow of the water is directly proportional to the square of the diameter. It takes 6 minutes for the smallest pipe to empty a full tank. F

In [7]:
json_ques = json.loads(output['output'])

str(list(json_ques.values())[0][0]).replace("'", '"')

'{"question": "A car traveling at a speed of 80 km/h uses 10 liters of fuel per hour. The car"s fuel consumption is directly proportional to its speed. How much fuel would the car use in an hour if it was traveling at a speed of 120 km/h?", "choices": {"a": "12 liters", "b": "15 liters", "c": "18 liters", "d": "20 liters"}, "answer": {"b": "15 liters"}}'

### Numerical MCQs

In [8]:
functions = [
    {
      "name": "get_results_from_wolfram_alpha",
      "description": """This function uses wolfram alpha endpoint that Understands natural language queries about entities in chemistry, physics, geography, history, art, astronomy, and more.\n- Performs mathematical calculations, date and unit conversions, formula solving, etc.\n- Convert inputs to simplified keyword queries whenever possible (e.g. convert \"how many people live in France\" to \"France population\").\n- Use ONLY single-letter variable names, with or without integer subscript (e.g., n, n1, n_1).\n- Use named physical constants (e.g., 'speed of light') without numerical substitution.\n- Include a space between compound units (e.g., \"Ω m\" for \"ohm*meter\").\n- To solve for a variable in an equation with units, consider solving a corresponding equation without units; exclude counting units (e.g., books), include genuine units (e.g., kg).""",
      "parameters": {
        "type": "object",
        "properties": {
          "queries": {
            "type": "string",
            "description": """curated natural language queries for wolfram alpha
            for example: "distance from earth to mars; current population of new delhi"."""
          }
        }
      },
        "required": ["queries"]
    },
    {
      "name": "get_results_from_wolfram_cloud",
      "description": """Use this function for problems solvable with Wolfram Language code.
      
      """,
      "parameters": {
        "type": "object",
        "properties": {
          "queries": {
            "type": "string",
            "description": """the input wolfram language query for mathematical calculations in string. ONLY proper wolfram language queries are supported.
            Format for Query:-
            "a = Solve[aCoeff1*aVar == Var1 - Offset1 && aCoeff2*aVar == Var1 + Offset2, {aVar, Var1}][[1, 1, 2]]".
            """
          }
        }
      },
        "required": ["queries"]
    },
] 

# Syntax & Code: Ensure syntactically correct Wolfram Language code. Remove comments and unnecessary formatting.
#       Functionality: Support complex calculations, data analysis, plotting, and data retrieval.
#       Entity Usage:
#       Before using Entity or similar, validate identifiers with Interpreter.
#       Prefer direct entity usage over typeData functions.
#       Code Composition:
#       Batch data retrieval when possible.
#       Organize data with Association.
#       Optimize for performance; minimize external calls.
#       Use camel case for variable names (e.g., variableName).
#       Enclose all strings in double quotes.
#       Handle unevaluated symbols with EntityValue.
#       Use Evaluate for complex expressions.
#       Response Formatting: All responses should be in Wolfram Language. Ensure clarity and accuracy.

# Example query: "x1 = Solve[0.30*x == PassingGrade - 50 && 0.70*x == PassingGrade + 20, {x, PassingGrade}][[1, 1, 2]]; y1 = Solve[0.20*y == RequiredCount - 40 && 0.60*y == RequiredCount + 30, {y, RequiredCount}][[1, 1, 2]]; z1 = Solve[0.35*z == RequiredScore - 65 && 0.75*z == RequiredScore + 35, {z, RequiredScore}][[1, 1, 2]]; {x1, y1, z1}"

In [9]:
# calculation check and improvements

def calculation_checking(input, model_id = "gpt-4-0613"):

    total_usage = {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}
    
    input_prompt = f"""

    INPUT: 

    {input}

    OUTPUT: 

    """ 
    
    system_message = {"role": "system", "content": CALCULATION_CHECK_PROMPT}
    conversation = [system_message]
    input_message = {'role': 'user', 'content': input_prompt}
    conversation.append(input_message)

    response = openai.ChatCompletion.create(
                                    model= model_id,
                                    messages = conversation,
                                    functions = functions,
                                    function_call = {'name': "get_results_from_wolfram_cloud"},
                                    temperature = 0
                                )

    total_usage = add_dicts(total_usage, dict(response.usage))
    print()
    print("AFTER FUNCTION", total_usage)
    print("calling function".upper())
    function_call = response.choices[0]['message']['function_call']
    conversation.append(response.choices[0]['message'])

    # print(response)

    # if function_call["name"] == "get_results_from_wolfram_cloud":
    queries = json.loads(function_call["arguments"])["queries"]
    print("QUERIES", queries)

    try:
        print("good queries", queries)
        result = get_results_from_wolfram_cloud(queries)
    except:
        raise Exception("error from wolfram")

# else:
#     raise "some error in function name"

    print("result from function".upper(), str(result))
    conversation.append({'role': 'function', "name": function_call["name"], "content": str(result)})

    response = openai.ChatCompletion.create(
        model = model_id,
        messages = conversation,
        functions = functions,
        temperature = 0.2
    )

    # print(response)

    # conversation.append(response.choices[0].message)
    total_usage = add_dicts(total_usage, dict(response.usage))
    print(total_usage)
    # return response, conversation
    
    # response, conversation = response_with_function(response, conversation)
    output = response.choices[0].message.content

    return {"output": output, "total_usage": total_usage}

# input = output['output']

correct_ques = []
total_usage = {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}
for ques in list(json_ques.values())[0]:
    ques = str(ques).replace("'", '"')
    result = calculation_checking(ques)
    total_usage = add_dicts(total_usage, result["total_usage"])
    correct_ques.append(result["output"])

print(correct_ques)


AFTER FUNCTION {'total_tokens': 738, 'completion_tokens': 32, 'prompt_tokens': 706}
CALLING FUNCTION
QUERIES fuel = Solve[80/10 == 120/fuel, fuel][[1, 1, 2]]
good queries fuel = Solve[80/10 == 120/fuel, fuel][[1, 1, 2]]
RESULT FROM FUNCTION 15
{'total_tokens': 1593, 'completion_tokens': 135, 'prompt_tokens': 1458}

AFTER FUNCTION {'total_tokens': 732, 'completion_tokens': 46, 'prompt_tokens': 686}
CALLING FUNCTION
QUERIES johnScore = 0.6*50; samScore = johnScore - 5; passingMarks = 0.5*50; {johnScore, samScore, passingMarks}
good queries johnScore = 0.6*50; samScore = johnScore - 5; passingMarks = 0.5*50; {johnScore, samScore, passingMarks}
RESULT FROM FUNCTION (30.0, 25.0, 25.0)
{'total_tokens': 1575, 'completion_tokens': 129, 'prompt_tokens': 1446}

AFTER FUNCTION {'total_tokens': 762, 'completion_tokens': 44, 'prompt_tokens': 718}
CALLING FUNCTION
QUERIES t = Solve[6 * (2^2) == t * ((2^2) + (4^2)), t][[1, 1, 2]]
good queries t = Solve[6 * (2^2) == t * ((2^2) + (4^2)), t][[1, 1, 2]]

In [10]:
print(correct_ques[0])


    {"question": "A car traveling at a speed of 80 km/h uses 10 liters of fuel per hour. The car"s fuel consumption is directly proportional to its speed. How much fuel would the car use in an hour if it was traveling at a speed of 120 km/h?", "choices": {"a": "12 liters", "b": "15 liters", "c": "18 liters", "d": "20 liters"}, "answer": {"b": "15 liters"}}


In [11]:
json.loads(correct_ques[0])

JSONDecodeError: Expecting ',' delimiter: line 2 column 98 (char 98)

In [ ]:
# ques with gpt-4-8k
for ques in correct_ques:
    jsonq = json.loads(ques)
    print(jsonq)

{'question': 'A farmer harvested 150 bushels of corn from 15 acres of his farm. If the yield of corn is directly proportional to the number of acres, how many bushels will he harvest from 30 acres?', 'choices': {'a': '100 bushels', 'b': '200 bushels', 'c': '300 bushels', 'd': '400 bushels'}, 'answer': {'c': '300 bushels'}}
{'question': 'In a Mathematics test, Tom scored 40% and missed by 20 marks to pass the test. Another student, Jack, who scored 50% marks, got 15 marks more than the minimum required marks to pass the examination. What is the maximum marks for the examination?', 'choices': {'a': '250', 'b': '175', 'c': '350', 'd': '400'}, 'answer': {'c': '350'}}
{'question': 'The travel time of a train is inversely proportional to its speed. If a train can cover a distance in 2 hours at 60 km/hr, how long will it take at 90 km/hr?', 'choices': {'a': '1.33 hours', 'b': '2.33 hours', 'c': '3.33 hours', 'd': '4.33 hours'}, 'answer': {'a': '1.33 hours'}}


In [ ]:
total_usage

{'completion_tokens': 403, 'total_tokens': 4778, 'prompt_tokens': 4375}

In [ ]:
calculate_cost_gpt4_8k(total_usage)

0.15543

In [ ]:
re = {'completion_tokens': 321, 'total_tokens': 877, 'prompt_tokens': 556}
total_tokens = add_dicts(re, total_usage)
calculate_cost_gpt4_8k(total_tokens) * 83.16

15.914329200000001

#### 18 rupees with gpt-4-8k

In [ ]:
total_usage

{'completion_tokens': 547, 'total_tokens': 6410, 'prompt_tokens': 5863}

In [ ]:
calculate_cost_gpt4_8k(total_usage)

0.20870999999999998

In [ ]:
# AFTER FUNCTION {'completion_tokens': 70, 'total_tokens': 1163, 'prompt_tokens': 1093}
# CALLING FUNCTION
# QUERIES d1 = 2; d2 = 4; d3 = 6; t1 = 5; t2 = t1 * (d1^2) / ((d1^2) + (d2^2) + (d3^2)); t2
# good queries d1 = 2; d2 = 4; d3 = 6; t1 = 5; t2 = t1 * (d1^2) / ((d1^2) + (d2^2) + (d3^2)); t2
# RESULT FROM FUNCTION Rational[5, 14]
# {'completion_tokens': 196, 'total_tokens': 2473, 'prompt_tokens': 2277}
# AFTER FUNCTION {'completion_tokens': 48, 'total_tokens': 1109, 'prompt_tokens': 1061}
# CALLING FUNCTION
# QUERIES a = Solve[0.3*x == y - 40 && 0.6*x == y + 10, {x, y}][[1, 1, 2]]
# good queries a = Solve[0.3*x == y - 40 && 0.6*x == y + 10, {x, y}][[1, 1, 2]]
# RESULT FROM FUNCTION 166.66666666666669
# {'completion_tokens': 132, 'total_tokens': 2322, 'prompt_tokens': 2190}
# AFTER FUNCTION {'completion_tokens': 80, 'total_tokens': 1176, 'prompt_tokens': 1096}
# CALLING FUNCTION
# QUERIES t1 = 8; d1 = 2; d2 = 4; d3 = 6; d4 = 8; t = t1 / ((d2^2/d1^2) + (d3^2/d1^2) + (d4^2/d1^2) + 1); t
# good queries t1 = 8; d1 = 2; d2 = 4; d3 = 6; d4 = 8; t = t1 / ((d2^2/d1^2) + (d3^2/d1^2) + (d4^2/d1^2) + 1); t
# RESULT FROM FUNCTION Rational[4, 15]
# {'completion_tokens': 210, 'total_tokens': 2503, 'prompt_tokens': 2293}
# ["{'question': 'The rate of flow of a pipe is directly proportional to the square of the diameter. If it takes 5 minutes for a pipe with a diameter of 2 cm to fill an empty tank, approximately how long would it take to fill the tank if all pipes of diameters 2 cm, 4 cm, and 6 cm are opened?', 'choices': {'a': '2 minutes', 'b': '1 minute', 'c': 'Approximately 0.36 minutes', 'd': '4 minutes'}, 'answer': {'c': 'Approximately 0.36 minutes'}}", "{'question': 'A student scores 30% and fails by 40 marks, while another student who scores 60% gets 10 marks more than the minimum required to pass. How many maximum marks are there for the examination?', 'choices': {'a': '250', 'b': '200', 'c': '300', 'd': '167'}, 'answer': {'d': '167'}}", "{'question': 'There are four inlet taps with diameters 2 cm, 4 cm, 6 cm, and 8 cm. The rate of flow of water is directly proportional to the square of the diameter. How long does it take to fill an empty tank when all four taps are opened, if it takes 8 minutes for the smallest pipe to fill the tank?', 'choices': {'a': '2 minutes', 'b': '4 minutes', 'c': '6 minutes', 'd': '8 minutes', 'e': '4/15 minutes'}, 'answer': {'e': '4/15 minutes'}}"]

In [ ]:
total_usage

{'completion_tokens': 538, 'total_tokens': 7298, 'prompt_tokens': 6760}

In [ ]:

calculate_cost_gpt4_8k(total_usage)
    

0.23480999999999996

0.040920000000000005

In [ ]:
# def get_results_from_wolfram_cloud(query):
#     from wolframclient.evaluation import WolframCloudSession, SecuredAuthenticationKey
#     from wolframclient.language import wl, wlexpr

#     key = SecuredAuthenticationKey(
#         os.environ.get('WOLFRAM_CLOUD_KEY1'),
#         os.environ.get('WOLFRAM_CLOUD_KEY2')
#     )

#     session = WolframCloudSession(credentials=key)

#     session.start()
#     result = session.evaluate(wlexpr(query))

#     return result
